In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras import models, layers, losses, optimizers, metrics

In [13]:
from keras.preprocessing.image import ImageDataGenerator
train_dir = 'data\\train'
test_dir ='data\\test'


train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (224, 224),
    batch_size=32
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (224, 224),
    batch_size=8
)

Found 189 images belonging to 7 classes.
Found 24 images belonging to 7 classes.


## Try using pretrined model

In [2]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(224, 224, 3))

conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Dense(7, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 3591      
Total params: 27,563,847
Trainable params: 27,563,847
Non-trainable params: 0
_________________________________________________________________


In [14]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.Adam(lr=5e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=6, 
                              epochs=30, 
                              validation_data=validation_generator, 
                              validation_steps=3)

This is the number of trainable weights before freezing the conv base: 4
Epoch 1/30
6/6 [==============================] - 7s 1s/step - loss: 2.1600 - categorical_accuracy: 0.2300 - val_loss: 1.9117 - val_categorical_accuracy: 0.2083
Epoch 2/30
6/6 [==============================] - 1s 233ms/step - loss: 1.7339 - categorical_accuracy: 0.3178 - val_loss: 1.7464 - val_categorical_accuracy: 0.2917
Epoch 3/30
6/6 [==============================] - 1s 233ms/step - loss: 1.4616 - categorical_accuracy: 0.4265 - val_loss: 1.6132 - val_categorical_accuracy: 0.4583
Epoch 4/30
6/6 [==============================] - 1s 232ms/step - loss: 1.1964 - categorical_accuracy: 0.6722 - val_loss: 1.5286 - val_categorical_accuracy: 0.3333
Epoch 5/30
6/6 [==============================] - 1s 233ms/step - loss: 1.0526 - categorical_accuracy: 0.7414 - val_loss: 1.4194 - val_categorical_accuracy: 0.5417
Epoch 6/30
6/6 [==============================] - 1s 232ms/step - loss: 0.9383 - categorical_accuracy: 0.7984 

In [46]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [15]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=6, 
                              epochs=50, 
                              validation_data=validation_generator, 
                              validation_steps=3)

Epoch 1/50
6/6 [==============================] - 2s 401ms/step - loss: 0.1228 - categorical_accuracy: 0.9948 - val_loss: 0.5743 - val_categorical_accuracy: 0.7917
Epoch 2/50
6/6 [==============================] - 1s 234ms/step - loss: 0.1253 - categorical_accuracy: 1.0000 - val_loss: 0.4670 - val_categorical_accuracy: 0.8750
Epoch 3/50
6/6 [==============================] - 1s 234ms/step - loss: 0.1086 - categorical_accuracy: 1.0000 - val_loss: 0.5265 - val_categorical_accuracy: 0.8750
Epoch 4/50
6/6 [==============================] - 1s 235ms/step - loss: 0.1121 - categorical_accuracy: 1.0000 - val_loss: 0.3808 - val_categorical_accuracy: 0.9167
Epoch 5/50
6/6 [==============================] - 1s 241ms/step - loss: 0.1093 - categorical_accuracy: 1.0000 - val_loss: 0.4522 - val_categorical_accuracy: 0.8750
Epoch 6/50
6/6 [==============================] - 1s 237ms/step - loss: 0.0990 - categorical_accuracy: 1.0000 - val_loss: 0.5026 - val_categorical_accuracy: 0.8333
Epoch 7/50
6/6 [

# Train the full VGG network

In [16]:
conv_base.trainable = True

model.compile(
    optimizer=optimizers.Adam(lr=1e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=6, 
                              epochs=50, 
                              validation_data=validation_generator, 
                              validation_steps=6)


Epoch 1/50
6/6 [==============================] - 11s 2s/step - loss: 0.0653 - categorical_accuracy: 0.9891 - val_loss: 0.3966 - val_categorical_accuracy: 0.8750
Epoch 2/50
6/6 [==============================] - 4s 632ms/step - loss: 0.0551 - categorical_accuracy: 0.9896 - val_loss: 0.3755 - val_categorical_accuracy: 0.7708
Epoch 3/50
6/6 [==============================] - 4s 626ms/step - loss: 0.0445 - categorical_accuracy: 0.9886 - val_loss: 0.4259 - val_categorical_accuracy: 0.8542
Epoch 4/50
6/6 [==============================] - 4s 628ms/step - loss: 0.0324 - categorical_accuracy: 0.9943 - val_loss: 0.3152 - val_categorical_accuracy: 0.8542
Epoch 5/50
6/6 [==============================] - 4s 626ms/step - loss: 0.0202 - categorical_accuracy: 0.9943 - val_loss: 0.4447 - val_categorical_accuracy: 0.8542
Epoch 6/50
6/6 [==============================] - 4s 637ms/step - loss: 0.0105 - categorical_accuracy: 1.0000 - val_loss: 0.2299 - val_categorical_accuracy: 0.8958
Epoch 7/50
6/6 [==

# Train longer with dropout

In [17]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(224, 224, 3))


model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(7, activation='softmax'))
model.summary()

conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.Adam(lr=5e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=6, 
                              epochs=15, 
                              validation_data=validation_generator, 
                              validation_steps=3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 3591      
Total params: 27,563,847
Trainable params: 27,563,847
Non-trainable params: 0
_________________________________________________________________
This is the number of trainable weights before freezing the conv base: 4
Epoch 1/15
6/6 [==============================] - 3s 534m

In [18]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
model.compile(
    optimizer=optimizers.Adam(lr=1e-5), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=6, 
                              epochs=50, 
                              validation_data=validation_generator, 
                              validation_steps=3)



Epoch 1/50
6/6 [==============================] - 3s 575ms/step - loss: 0.2965 - categorical_accuracy: 0.9791 - val_loss: 0.7236 - val_categorical_accuracy: 0.8333
Epoch 2/50
6/6 [==============================] - 2s 268ms/step - loss: 0.2168 - categorical_accuracy: 0.9843 - val_loss: 0.4334 - val_categorical_accuracy: 0.8750
Epoch 3/50
6/6 [==============================] - 2s 271ms/step - loss: 0.1524 - categorical_accuracy: 0.9843 - val_loss: 0.5471 - val_categorical_accuracy: 0.7917
Epoch 4/50
6/6 [==============================] - 2s 276ms/step - loss: 0.1295 - categorical_accuracy: 0.9948 - val_loss: 0.2983 - val_categorical_accuracy: 0.9583
Epoch 5/50
6/6 [==============================] - 2s 276ms/step - loss: 0.1192 - categorical_accuracy: 0.9948 - val_loss: 0.3332 - val_categorical_accuracy: 0.8750
Epoch 6/50
6/6 [==============================] - 2s 272ms/step - loss: 0.0811 - categorical_accuracy: 0.9948 - val_loss: 0.2706 - val_categorical_accuracy: 0.9583
Epoch 7/50
6/6 [